In [ ]:
from ultralytics import YOLO
import cv2

model = YOLO(r"E:\best.pt")
results = model.predict(source="0", show=True)
print(results)

In [ ]:
import cv2
import torch
from ultralytics import YOLO
import socket

# Check if GPU is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set up the socket for communication with ESP32
HOST = 'http://192.168.137.236/data'  # IP address of the ESP32
PORT = 80  # Port for communication
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))

# URL for the stream
stream_url = "http://192.168.137.94:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Collect results and send labels to ESP32
results_count = 0
while results_count < 10:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model.predict(resized_frame, show=True)

    # Extract label from results and send it to ESP32
    label_id = results.names[0]
    label = 'P'  # Default label is 'p' (plastic) for demonstration purposes
    if label_id == 3:  # Assuming 'Plastic' has label ID 3
        label = 'p'
    elif label_id == 0:  # Assuming 'Biodegradable' has label ID 0
        label = 'b'
    elif label_id == 1:  # Assuming 'Glass' has label ID 1
        label = 'g'

    # Send label to ESP32
    sock.sendall(label.encode())

    # Increment the results count
    results_count += 1

# Release the video capture object, close the socket, and close the window
cap.release()
sock.close()
cv2.destroyAllWindows()


In [ ]:
import cv2
import torch
from ultralytics import YOLO
import requests

# Function to send command to ESP32
def send_command(command):
    data = command + '\n'
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0

# ESP IP URL
url = 'http://192.168.150.99/data'

# URL for the stream
stream_url = "http://100.86.205.220:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Collect results and send labels to ESP32
results_count = 0
while results_count < 10:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model.predict(resized_frame, show=True)

    for result in results:
        for prediction in result:
        # Check if the prediction is not an empty list and contains necessary elements
            if len(prediction) >= 6:  # Check the minimum length required
                label_id = prediction[5]  # Accessing the class ID
                confidence = prediction[4]  # Accessing the confidence score

                # Logic to assign labels based on label IDs

                # Assuming 'Plastic' has label ID 3
                # Assuming 'Biodegradable' has label ID 0
                # Assuming 'Glass' has label ID 1
                # Assuming 'Metal' has label ID 2

                label = 'P'  # Default label is 'P' (plastic) for demonstration purposes
                if label_id == 3:
                    label = 'P'
                elif label_id == 0:
                    label = 'B'
                elif label_id == 1:
                    label = 'G'
                elif label_id == 2:
                    label = 'O'

                # Send label to ESP32
                send_command(label)

                # Increment the results count for each detection
                results_count += 1
            else:
                print("Invalid prediction format: ", prediction)

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
import socket
import random

# ESP32 IP and port
esp32_ip = "10.42.0.1"  # Replace with your ESP32's IP address
esp32_port = 80

# Create a socket object
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect to the ESP32
client_socket.connect((esp32_ip, esp32_port))

# Send random values to the ESP32
for _ in range(5):
    data_to_send = str(random.randint(1, 100))  # Generate a random value
    client_socket.sendall(data_to_send.encode())
    print("Sent data to ESP32:", data_to_send)

# Close the socket
client_socket.close()


In [ ]:
import cv2
from ultralytics import YOLO

# URL for the stream
stream_url = "http://192.168.137.94:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Read and display the video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640,480))

    # Perform object detection on the resized frame
    results = model.predict(resized_frame, show=True)
    print(results)

    # Display the original frame
    cv2.imshow('Frame', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO
model = YOLO(r"E:\best.pt")
model.export(format="saved_model")

In [ ]:
import cv2

# Initialize the camera
cap = cv2.VideoCapture(0)  # Use 0 for the default camera or specify the camera index

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Display the frame
    cv2.imshow("Camera Stream", frame)

    # Check for a key press and break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Exit the loop when 'q' is pressed

# Release the camera and close OpenCV windows
cap.release()
cv2.destroyAllWindows()
